<a href="https://colab.research.google.com/github/GajananKuchgave/EVA8_Assignments/blob/main/Assignment_3/EVA8_Session_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        #self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16,16, 3,stride =2, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.Dp1 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(16, 32, 3, stride =2, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.Dp2 = nn.Dropout(0.25)
        self.conv4 = nn.Conv2d(32, 10, 3,stride =2, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.conv5 = nn.Conv2d(256, 512, 3)
        #self.bn5 = nn.BatchNorm2d(512)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.bn6 = nn.BatchNorm2d(1024)
        #self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.Dp1(self.pool1(self.bn2(F.relu(self.conv2(F.relu(self.conv1(x)))))))
        x = self.pool2(F.relu(self.conv3(x)))
        #x = self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = self.conv4(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
            Conv2d-2           [-1, 16, 14, 14]           2,320
       BatchNorm2d-3           [-1, 16, 14, 14]              32
         MaxPool2d-4             [-1, 16, 7, 7]               0
           Dropout-5             [-1, 16, 7, 7]               0
            Conv2d-6             [-1, 32, 4, 4]           4,640
         MaxPool2d-7             [-1, 32, 2, 2]               0
            Conv2d-8             [-1, 10, 1, 1]           2,890
Total params: 10,042
Trainable params: 10,042
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.16
Params size (MB): 0.04
Estimated Total Size (MB): 0.20
----------

In [5]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [15]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    Accuracy = float(100. * correct / len(test_loader.dataset))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        Accuracy))

In [16]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.08691071718931198 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.37it/s]



Test set: Average loss: 0.0732, Accuracy: 9775/10000 (97.75%)



loss=0.03662353754043579 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 33.14it/s]



Test set: Average loss: 0.0520, Accuracy: 9830/10000 (98.30%)



loss=0.09647294133901596 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 33.50it/s]


...


Test set: Average loss: 0.0291, Accuracy: 9908/10000 (99.08%)



loss=0.02550884336233139 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.81it/s]



Test set: Average loss: 0.0325, Accuracy: 9900/10000 (99.00%)



loss=0.0670229122042656 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.94it/s]



Test set: Average loss: 0.0281, Accuracy: 9912/10000 (99.12%)

